In [ ]:
%pip install finta
%pip install yfinance
%pip install backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
import requests
import json
from finta import TA
import pandas as pd

In [ ]:
def create_price_data_dict(symbols, start_date, end_date):
    dict = {}
    
    for symbol in symbols:
        url = f'https://api.pro.coinbase.com/products/{symbol}-USD/candles'
        params = {'start': start_date, 'end': end_date, 'granularity': 3600}
        response = requests.get(url, params=params)
        data = json.loads(response.text)

        df = pd.DataFrame(data, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
        df['Date'] = pd.to_datetime(df['Date'], unit='s')
        df.set_index('Date', inplace=True)
        df.sort_index(inplace=True)
        
        dict[symbol] = df

    return dict

symbols = ['BTC', 'ETH', 'LINK', 'ADA', 'DOGE']
start_date = '2023-04-07T00:00:00.000Z'
end_date = '2023-04-18T23:59:59.999Z'

price_data_dict = create_price_data_dict(symbols, start_date, end_date)
price_data_dict

In [120]:
class SmaCrossWithRSI(Strategy):
    # Initialize variables
    short_sma = 20
    long_sma = 50
    atr_length = 14
    rsi_length = 14

    # Initialize the strategy. Instances are stored as attributes in our SmaCrossWithRSI object. 
    def init(self):
        # Creates pandas series of the price data for later use. 
        open = pd.Series(self.data.Open)
        close = pd.Series(self.data.Close)
        high = pd.Series(self.data.High)
        low = pd.Series(self.data.Low)
        volume = pd.Series(self.data.Volume)

        # Creates df for the ohlcv data to calculate ATR and RSI.
        ohlcv = pd.DataFrame({'open': open, 'high': high, 'low': low, 'close': close, 'volume': volume})

        # Create instance of ATR and RSI indicators from finta, and the native SMA indicator from bt.
        self.atr = self.I(TA.ATR, ohlcv, self.atr_length)
        self.rsi = self.I(TA.RSI, ohlcv, self.rsi_length)
        self.sma1 = self.I(SMA, close, self.short_sma)
        self.sma2 = self.I(SMA, close, self.long_sma)

    # Logic for trade entrance and exit
    def next(self):
        # Price set to the candles 'close' value, used for calculating stop loss and take profit targets.
        price = self.data.Close

        # ATR and RSI for the latest candle.
        ATR = self.atr[-1]
        RSI = self.rsi[-1]

        # If sma1 crosses over sma2, and RSI > 50, then enter a long position with the desired target logic.
        if crossover(self.sma1, self.sma2) and RSI > 50:
            stop_loss = price - (1.1 * ATR)
            take_profit = price + (2 * ATR)
            self.buy(sl=stop_loss, tp=take_profit)

In [ ]:
def run_backtests(price_data_dict, strategy_class, cash=100000, commission=0.002, exclusive_orders=True):
    backtest_results = {}
    
    for symbol, data in price_data_dict.items():
        bt = Backtest(data, strategy_class, cash=cash, commission=commission, exclusive_orders=exclusive_orders)
        strategy_trade_results = bt.run()
        strategy_trade_results_df = pd.DataFrame(strategy_trade_results).transpose()
        backtest_results[symbol] = strategy_trade_results_df

    return backtest_results

backtest_results = run_backtests(price_data_dict, SmaCrossWithRSI)
backtest_results

In [151]:
# 1. List the metrics you want to extract
metrics_to_extract = ['# Trades', 'Return [%]', 'Sharpe Ratio', 'Win Rate [%]']

# 2. Extract the metrics from the individual DataFrames
metrics_data = {}

for symbol, backtest_result_df in backtest_results.items():
    metrics_data[symbol] = backtest_result_df.loc[0, metrics_to_extract]

# 3. Combine the extracted metrics into a single DataFrame
combined_metrics_df = pd.DataFrame(metrics_data).transpose()
combined_metrics_df['Symbols'] = combined_metrics_df.index
combined_metrics_df.set_index('Symbols', inplace=True)

print(combined_metrics_df)

        # Trades Return [%] Sharpe Ratio Win Rate [%]
Symbols                                              
BTC            3   0.006898     0.037013    33.333333
ETH            4   2.361414      4.77247         75.0
LINK           2  -0.442156          0.0         50.0
ADA            2  -1.174481          0.0          0.0
DOGE           2   2.506253     5.216987        100.0


In [160]:
# Calculate the sum and mean
total_trades = combined_metrics_df['# Trades'].sum()
mean_return = combined_metrics_df['Return [%]'].mean()
mean_sharpe_ratio = combined_metrics_df['Sharpe Ratio'].mean()
mean_win_rate = combined_metrics_df['Win Rate [%]'].mean()

# Create a DataFrame with the aggregated data
aggregated_data = pd.DataFrame(data=[[total_trades, mean_return, mean_sharpe_ratio, mean_win_rate]], columns=metrics_to_extract)
# Set '# Trades' as the index
aggregated_data.set_index('# Trades', inplace=True)
print(aggregated_data)

          Return [%]  Sharpe Ratio  Win Rate [%]
# Trades                                        
13          0.651586      2.005294     51.666667
